<a href="https://colab.research.google.com/github/arunpras/Code-Sample/blob/main/get_costco_gold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install fake-useragent


In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from fake_useragent import UserAgent

# Define your search keywords and negative keywords
keywords = ['gold bar', 'gold coin', 'silver coin', 'silver bar']
negative_keywords = ['necklace', 'plated', 'coated', 'search', 'keyword', 'pendant', 'baguette']  # Add 'search' to your negative keywords
no_match_phrase = "We were not able to find a match"  # Phrase indicating no matches

def get_valid_urls(soup, keyword):
    """Filter URLs based on negative keywords and the presence of search keywords."""
    valid_urls = []  # Initialize a list to hold valid URLs
    # Find all links in the search result
    for link in soup.find_all('a', href=True):
        url = urljoin("https://www.costco.com", link['href'])
        # Exclude URLs containing any negative keywords
        if not any(neg_kw in url for neg_kw in negative_keywords):
            # Include URL if it contains all parts of the search keyword
            if all(word.lower() in url.lower() for word in keyword.split()):
                valid_urls.append(url)
    return valid_urls

def search_costco(keyword, no_match_phrase):
    """Searches Costco.com for the keyword."""
    ua = UserAgent()  # Initialize UserAgent object
    headers = {'User-Agent': ua.random}  # Generate a random User-Agent
    search_url = f'https://www.costco.com/CatalogSearch?dept=All&keyword="%22{keyword.replace(" ", "+")}%22"'
    response = requests.get(search_url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        if no_match_phrase.lower() in response.text.lower():
            print(f'"{keyword}": {no_match_phrase}')
            return []  # Early return if no match phrase is found
        else:
            return get_valid_urls(soup, keyword)
    else:
        print(f"Failed to access {search_url}: Status code {response.status_code}")
        return []

def main(no_match_phrase):
    for keyword in keywords:
        print(f'\nSearching Costco.com for "{keyword}"...')
        valid_urls = search_costco(keyword, no_match_phrase)
        if valid_urls:
            print(f"Valid URLs for '{keyword}':")
            for url in valid_urls:
                print(url)
        else:
            print(f"No products found for '{keyword}' that meet the criteria or '{no_match_phrase}' was found.")

if __name__ == "__main__":
    main(no_match_phrase)



Searching Costco.com for "gold bar"...
"gold bar": We were not able to find a match
No products found for 'gold bar' that meet the criteria or 'We were not able to find a match' was found.

Searching Costco.com for "gold coin"...
"gold coin": We were not able to find a match
No products found for 'gold coin' that meet the criteria or 'We were not able to find a match' was found.

Searching Costco.com for "silver coin"...
"silver coin": We were not able to find a match
No products found for 'silver coin' that meet the criteria or 'We were not able to find a match' was found.

Searching Costco.com for "silver bar"...
"silver bar": We were not able to find a match
No products found for 'silver bar' that meet the criteria or 'We were not able to find a match' was found.


In [5]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs, unquote
from fake_useragent import UserAgent

keywords = ['gold bar', 'gold coin', 'silver coin', 'silver bar']
search_words = ['product not available']

def clean_url(url):
    """Extract and clean URL from Google's redirect URL if necessary and ensure it points to a Costco product page."""
    if url.startswith('/url?q='):
        parsed_url = urlparse(url)
        extracted_url = parse_qs(parsed_url.query).get('q', [''])[0]
        cleaned_url = unquote(extracted_url)
        # Ensure the URL is for Costco and does not point to Google search results or maps
        if 'costco.com' in cleaned_url and not cleaned_url.startswith('https://maps.google.com') \
           and 'accounts.google.com' not in cleaned_url and not cleaned_url.startswith('/search'):
            return cleaned_url
    return None


def is_keyword_in_url(url, keywords):
    """Check if the URL contains any of the keyword phrases."""
    for keyword in keywords:
        # Split the keyword into individual words and check if all are in the URL
        if all(word in url for word in keyword.split()):
            return True
    return False

def search_google(query):
    """Searches Google and returns Costco links that match the keyword criteria. Filters out non-product links."""
    headers = {'User-Agent': 'Mozilla/5.0'}
    search_url = f'https://www.google.com/search?q=site:costco.com+{query}'
    response = requests.get(search_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a')
    costco_links = []
    for link in links:
        href = link.get('href')
        if href:
            cleaned_url = clean_url(href)
            # Check if the cleaned URL is not None and matches the keyword criteria
            if cleaned_url and is_keyword_in_url(cleaned_url, keywords):
                costco_links.append(cleaned_url)
    return costco_links

def check_words_in_page(url, search_words):
    """Check if any of the specified words exist in the web page at the given URL."""
    try:
        # Initialize UserAgent object
        ua = UserAgent()

        # Generate a random User-Agent
        headers = {'User-Agent': ua.random}

        # Adding a timeout to the request and passing the random User-Agent
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            # Uncomment the next line if you need to debug or inspect the response text
            # print(f'Found the following text, \n {response.text}')
            soup = BeautifulSoup(response.text, 'html.parser')
            text = soup.get_text().lower()
            return any(word.lower() in text for word in search_words)
        else:
            print(f"Failed to access {url}: Status code {response.status_code}")
            return False
    except requests.exceptions.Timeout:
        # If a timeout occurs
        print(f"Request to {url} timed out.")
        return False
    except Exception as e:
        print(f"An error occurred while accessing {url}: {e}")
        return Falseg

def main():
    available_products = []  # List to hold URLs of available products

    for keyword in keywords:
        print(f'Searching for "{keyword}"...')
        # Now search_google uses the keyword directly for more accurate results
        costco_links = search_google(keyword)
        if costco_links:
            print(f'Found {len(costco_links)} valid links for "{keyword}".')
            for link in costco_links:
                print(f'Checking link: {link}')
                exists = check_words_in_page(link, search_words)
                if not exists:  # If 'words found' is False, the product is considered available
                    available_products.append(link)  # Add to the list of available products
                    print(f"Product available at: {link}")
                # else:
                #     print(f"Product not available or page contains search words: {link}")
        else:
            print(f'No valid Costco links found for "{keyword}".')

    # After processing all keywords, print or process the list of available product URLs
    if available_products:
        print("\nList of available product URLs:")
        for product_url in available_products:
            print(product_url)
    else:
        print("No available products found.")

if __name__ == "__main__":
    main()

Searching for "gold bar"...
Found 10 valid links for "gold bar".
Checking link: https://www.costco.com/1-oz-gold-bar-pamp-suisse-lady-fortuna-veriscan-new-in-assay.product.4000186760.html
Checking link: https://www.costco.com/1-oz-gold-bar-pamp-suisse-diwali-lakshmi.product.4000300302.html
Checking link: https://www.costco.com/1-oz-gold-bar-rand-refinery-new-in-assay.product.4000202779.html
Checking link: https://www.costco.com/50-gram-gold-bar-pamp-suisse-lady-fortuna-new-in-assay.product.4000226964.html
Checking link: https://www.costco.com/100-gram-gold-bar-pamp-rosa-new-in-assay.product.4000227041.html
Checking link: https://www.costco.com/1-oz-gold-bar-pamp-rosa-new-in-assay.product.4000226984.html
Checking link: https://www.costco.com/25-gram-pamp-suisse-lady-fortuna-multigram-gold-bar-veriscan-new-in-assay.product.4000249484.html
Checking link: https://www.costco.com/100-gram-gold-bar-pamp-suisse-lady-fortuna-veriscan-new-in-assay.product.4000239966.html
Checking link: https://w